In [9]:
""" import necessary libraries """

import os
import soundfile as sf
import librosa
import random
import soundfile as sf
import numpy as np


from pydub import AudioSegment


In [10]:
# user-specific variables
char_path = 'F:/ZaknafeinII_Backup_02-02-22/daea/prepareForCtnTraining/practice/exp_char/'
noisepool_path = 'F:/ZaknafeinII_Backup_02-02-22/daea/prepareForCtnTraining/practice/exp_noisepool/'
output_path = 'F:/ZaknafeinII_Backup_02-02-22/daea/prepareForCtnTraining/practice/output/'

# the max number of noise clips to be overlayed over the char clip
# for each char clip, a random number from 1 to max_overlays will be overlayed
max_overlays = 3

In [22]:
# iterable object creation
char_clip_names = os.listdir(char_path)
noisepool_dirs = os.listdir(noisepool_path)
print(noisepool_dirs)

for char_clip_name in char_clip_names:
    print(char_clip_name)
    
    char_clip = librosa.load(char_path + char_clip_name)
    
    # randomly determine how many noise clips to overlay (inclusive, inclusive)
    to_overlay = random.randint(1, max_overlays)
    
    # iterate equal to to_overlay to build the select the noise clips
    noise_clips = []
    noisepool_dirs_used = []
    savename = ""
    for i in range(0, to_overlay):
        # randomly select a noise character
        # cannot be a character that we have already used
        # because one person cannot speak over themselves in a conversation
        selected_dir = None
        while selected_dir == None or selected_dir in noisepool_dirs_used:
            selected_dir = noisepool_dirs[random.randrange(0, len(noisepool_dirs))]
        noisepool_dirs_used.append(selected_dir)
        
        # now we randomly select a clip from that noise character's dir
        selected_dir_clips = os.listdir(noisepool_path + selected_dir)
        selected_clip_name = selected_dir_clips[random.randrange(0, len(selected_dir_clips))]
        selected_clip, _ = librosa.load(noisepool_path + selected_dir + '/' + selected_clip_name)
        
        noise_clips.append(selected_clip)
        savename += '_' + selected_clip_name[:-4]

    merged_noise_clip = np.array(sum(noise_clips)/1)
    # save the selected clip to file
    bool_ = sf.write(output_path + str(to_overlay) + savename + '.wav', merged_noise_clip, 22050)
    print('write done:', bool_)
#     merged_clip = 
    
    
    
#     # number of frames = 4 minutes * 60 seconds * 22050 frames per second 
#     summed_duration = 0
#     # merged_wav = None
    
#     curr_char_dir = pc_specific_path + '/' + character_id
#     #print(curr_char_dir)
#     # iterate over each audio directory belonging to that character
    
#     char_specific_clips = os.listdir(curr_char_dir)
#     for clip_folder in char_specific_clips:
#         #print(clip_folder)
#         clips = os.listdir(curr_char_dir + '/' + clip_folder)
#         for clip in clips:
#             clip_filepath = curr_char_dir + '/' + clip_folder + '/' + clip
#             #print(clip_filepath)
#             audio = AudioSegment.from_file("C:/Users/eholy/Desktop/SD/vox2/vox2_dev_aac_partaa~/dev/aac/id00029/96Z18I2UKAE/00001.m4a").set_frame_rate(22050)
#             audio_array = audio.get_array_of_samples()
#             summed_duration = summed_duration + len(audio_array)
#             if summed_duration > 5292000:
#                 id_over_4.append(curr_char_dir)
#                 break
#         #print(summed_duration)
#             #sf.write(output_path + '/' + 'test.wav', audio_array, 22050)
#             #print(audio_array)
            
        
# print(len(id_over_4))
        
        
            

['id10016_merged', 'id10018_merged', 'id10020_merged']
id10045_merged_0-88200.wav
write done: None
id10045_merged_104728-192928.wav
write done: None
id10045_merged_203944-292144.wav
write done: None
id10045_merged_303160-391360.wav
write done: None
id10045_merged_402376-490576.wav
write done: None
id10045_merged_501592-589792.wav
write done: None
id10045_merged_600808-689008.wav
write done: None
id10045_merged_700024-788224.wav
write done: None
id10045_merged_804752-892952.wav
write done: None
